In [ ]:
!git clone https://github.com/hyphacoop/organizing.git

fatal: destination path 'organizing' already exists and is not an empty directory.


In [1]:
import os
import json
import markdown
from bs4 import BeautifulSoup

In [2]:
output=[]
md_dir = "organizing/resolutions/"
if os.path.exists(md_dir):
  for fname in os.listdir(md_dir):
    if fname.endswith(".md"):
      with open(os.path.join(md_dir, fname), 'r') as f:
        raw_md = f.read()
        html = markdown.markdown(raw_md)
        text = BeautifulSoup(html, 'html.parser').get_text()
        output.append({
            "instruction": "Generate a board resolution similar to: " + fname.replace("-", " ").replace(".md", ""),
            "response": text.strip()
        })

os.makedirs("data", exist_ok=True)
with open("data/board_resolution.jsonl", "w") as f:
  for entry in output:
    f.write(json.dumps(entry)+"\n")

print(f"✅ Saved {len(output)} examples to data/board_resolutions.jsonl")



✅ Saved 0 examples to data/board_resolutions.jsonl


In [3]:
import os

md_dir = "organizing/resolutions"
print(f"Looking in: {md_dir}")
print("Files found:", os.listdir(md_dir) if os.path.exists(md_dir) else "Directory not found")


Looking in: organizing/resolutions
Files found: Directory not found


In [4]:
def convert_jsonl(input_path, output_path):
  records=[]
  with open(input_path, 'r', encoding='utf-8') as f:
    for line in f:
      data = json.loads(line)
      record = {"instruction": data.get("instruction", ""),
                "input": data.get("input", ""),
              "output": data.get("response", data.get("output", ""))
              }
      records.append(record)
  with open(output_path, 'w', encoding='utf-8') as f:
    for record in records:
      f.write(json.dumps(record)+"\n")

In [5]:
output_path = "data/output"
os.mkdir(output_path)


In [6]:
convert_jsonl("data/board_resolution.jsonl", os.path.join(output_path, "board_resolution.jsonl"))

In [ ]:
from datasets import Dataset
import json

# Read the peft-formatted file
with open("data/output/board_resolution.jsonl", "r", encoding="utf-8") as f:
    records = [json.loads(line) for line in f]

# Create a Dataset
ds = Dataset.from_list(records)




In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login

login()  # Will ask for your token



In [ ]:
from huggingface_hub import snapshot_download

# This will only download the original files (sharded .safetensors) without merging
model_path = snapshot_download(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    cache_dir="/content/drive/MyDrive/hf_models/mistral",
    allow_patterns=[
        "config.json",
        "generation_config.json",
        "tokenizer.json",
        "tokenizer_config.json",
        "tokenizer.model",
        "special_tokens_map.json",
        "model-00001-of-00003.safetensors",
        "model-00002-of-00003.safetensors",
        "model-00003-of-00003.safetensors",
        "model.safetensors.index.json"
    ]
)

print(f"Model files downloaded to: {model_path}")


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

Model files downloaded to: /content/drive/MyDrive/hf_models/mistral/models--mistralai--Mistral-7B-Instruct-v0.3/snapshots/e0bc86c23ce5aae1db576c8cca6f06f1f73af2db


In [4]:
!pip install -U bitsandbytes

In [10]:
from transformers import AutoModelForCausalLM, LlamaTokenizer, BitsAndBytesConfig
import os

# Find the actual model path inside snapshots
base_dir = "/content/drive/MyDrive/hf_models/mistral/models--mistralai--Mistral-7B-Instruct-v0.3"
snapshots_dir = os.path.join(base_dir, "snapshots")
snapshot_subdir = os.listdir(snapshots_dir)[0]
model_path = os.path.join(snapshots_dir, snapshot_subdir)

# Quantization config with CPU offload allowed
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
    llm_int8_enable_fp32_cpu_offload=True  # ✅ required when some modules go to CPU
)

# Load tokenizer
tokenizer = LlamaTokenizer.from_pretrained(model_path, legacy=True)

# Load model with CPU offloading enabled
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",                # Let HF decide placement
    quantization_config=bnb_config   # With offloading support
)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
import json

jsonl_path ="/content/drive/MyDrive/dataset_lexgen/board_resolution.jsonl"

data = []
with open(jsonl_path, "r") as f:
    for line in f:
        data.append(json.loads(line))

print(f"✅ Loaded {len(data)} examples")
print(data[0])
#added few generated examples

✅ Loaded 25 examples
{'instruction': 'Generate a board resolution similar to: resolution 004', 'input': '', 'output': 'RESOLUTIONS OF THE BOARD OF DIRECTORS\nResolution in writing of the board of directors of Hypha Worker Co-operative Inc. (the "Co-operative") held on this 10th day of October, 2021.\nBACKGROUND\n\n\nWe have surplus capital to allocate within the Co-operative.\n\n\nPatrick Cahill Connolly stepped down as vice president and director of the Co-operative as of the 26th day of May, 2021.\n\n\nIT WAS RESOLVED THAT\n\n\nThe directors unanimously approve of allocating 80% of surplus capital to reserve funds and 20% to member capital accounts.\n\n\nThe position of vice president remain vacant.\n\n\nCURRENT STATE OF THE CO-OPERATIVE\nDirectors: Yurko Jaremkο, Benedict Lau, Udit Vira, Elon Li, Dawn Walker\nOfficers:\n* President: Yurko Jaremkο\n* Vice president: vacant\n* Secretary: Benedict Lau\n* Treasurer: Udit Vira\nThis resolution may be executed in counterparts. Facsimile, 

In [12]:
from datasets import Dataset

# Merge instruction + input as the full prompt
formatted_data = [
    {
        "text": f"<s>[INST] {example['instruction']}\n\n{example.get('input', '')} [/INST] {example['output']}</s>"
    }
    for example in data  # or use dataset if you're using load_dataset
]

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_list(formatted_data)

# Preview first sample
print(train_dataset[0])


{'text': '<s>[INST] Generate a board resolution similar to: resolution 004\n\n [/INST] RESOLUTIONS OF THE BOARD OF DIRECTORS\nResolution in writing of the board of directors of Hypha Worker Co-operative Inc. (the "Co-operative") held on this 10th day of October, 2021.\nBACKGROUND\n\n\nWe have surplus capital to allocate within the Co-operative.\n\n\nPatrick Cahill Connolly stepped down as vice president and director of the Co-operative as of the 26th day of May, 2021.\n\n\nIT WAS RESOLVED THAT\n\n\nThe directors unanimously approve of allocating 80% of surplus capital to reserve funds and 20% to member capital accounts.\n\n\nThe position of vice president remain vacant.\n\n\nCURRENT STATE OF THE CO-OPERATIVE\nDirectors: Yurko Jaremkο, Benedict Lau, Udit Vira, Elon Li, Dawn Walker\nOfficers:\n* President: Yurko Jaremkο\n* Vice president: vacant\n* Secretary: Benedict Lau\n* Treasurer: Udit Vira\nThis resolution may be executed in counterparts. Facsimile, scanned, or digital signatures a

In [13]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()



trainable params: 3,407,872 || all params: 7,251,431,424 || trainable%: 0.0470


In [29]:


# tell it to treat the EOS as padding
tokenizer.pad_token = tokenizer.eos_token

# now your `.map(tokenize, …)` will pad against EOS


def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

tokenized_dataset = train_dataset.map(tokenize, batched=True)

print(tokenized_dataset[0])


Map:   0%|          | 0/25 [00:00<?, ? examples/s]

{'text': '<s>[INST] Generate a board resolution similar to: resolution 004\n\n [/INST] RESOLUTIONS OF THE BOARD OF DIRECTORS\nResolution in writing of the board of directors of Hypha Worker Co-operative Inc. (the "Co-operative") held on this 10th day of October, 2021.\nBACKGROUND\n\n\nWe have surplus capital to allocate within the Co-operative.\n\n\nPatrick Cahill Connolly stepped down as vice president and director of the Co-operative as of the 26th day of May, 2021.\n\n\nIT WAS RESOLVED THAT\n\n\nThe directors unanimously approve of allocating 80% of surplus capital to reserve funds and 20% to member capital accounts.\n\n\nThe position of vice president remain vacant.\n\n\nCURRENT STATE OF THE CO-OPERATIVE\nDirectors: Yurko Jaremkο, Benedict Lau, Udit Vira, Elon Li, Dawn Walker\nOfficers:\n* President: Yurko Jaremkο\n* Vice president: vacant\n* Secretary: Benedict Lau\n* Treasurer: Udit Vira\nThis resolution may be executed in counterparts. Facsimile, scanned, or digital signatures a

In [30]:
# After tokenization, the dataset has `input_ids` & `attention_mask`.
# Add a 'labels' column that duplicates 'input_ids'.
def add_labels(examples):
    examples["labels"] = examples["input_ids"].copy()
    return examples

tokenized_dataset = tokenized_dataset.map(add_labels, batched=True)

# Now tokenized_dataset columns: ['input_ids', 'attention_mask', 'labels']

# Data collator for causal LM
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Trainer setup (no renaming needed)
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/mistral_lexgen_lora_output",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    max_steps=200,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="steps",
    save_steps=100,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

# training:
trainer.train()


Map:   0%|          | 0/25 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  

Step,Training Loss
10,2.210200
20,1.623700
30,1.154000
40,0.877600
50,0.556100
60,0.393900
70,0.299200
80,0.215600
90,0.200200
100,0.166900


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


TrainOutput(global_step=200, training_loss=0.44802395582199095, metrics={'train_runtime': 3650.4145, 'train_samples_per_second': 0.438, 'train_steps_per_second': 0.055, 'total_flos': 2.733010059264e+16, 'train_loss': 0.44802395582199095, 'epoch': 50.0})

In [7]:



import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

MODEL_PATH   = "/content/drive/MyDrive/hf_models/mistral/models--mistralai--Mistral-7B-Instruct-v0.3/snapshots/e0bc86c23ce5aae1db576c8cca6f06f1f73af2db"
ADAPTER_PATH = "/content/drive/MyDrive/mistral_lexgen_lora_output/checkpoint-200"
OFFLOAD_DIR  = "/content/bnb_offload"

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True,
    llm_int8_threshold=6.0,
    offload_folder=OFFLOAD_DIR
)

 #Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True,
    local_files_only=True
)

#Load Base Model (8‑bit + offload)
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    offload_folder="offload",
    offload_state_dict=True,
    local_files_only=True
)

# Apply LoRA Adapter (offload-enabled)
model = PeftModel.from_pretrained(
    base_model,
    ADAPTER_PATH,
    is_trainable=False,
    adapter_name="default",  # helps in avoiding mismatched keys sometimes
)

model.eval()

# Inference Function
def generate_resolution(instruction, user_input, max_new_tokens=256):
    prompt = f"<s>[INST] {instruction}\n\n{user_input} [/INST]"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        top_p=0.9,
        do_sample=True
    )
    return tokenizer.decode(out[0][inputs.input_ids.shape[-1]:], skip_special_tokens=True)

#  Quick Test
instr = "Generate a board resolution for the given input."
ui    = "Company Name: Aretoss Pvt Ltd\nDate: 1 June 2025\nPurpose: Appoint CFO"
print(generate_resolution(instr, ui))


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 RESOLVED THAT Mr. Vineet Shenoy be and is hereby appointed as the CFO (Chief Financial Officer) of Aretoss Pvt Ltd with effect from 1 June 2025.

FURTHER RESOLVED THAT Mr. Shenoy's appointment letter be prepared and signed by the Managing Director.

LASTLY RESOLVED THAT a copy of this resolution be placed on record and the CFO may also be provided with a copy of this resolution.

__________ has initiated the AlgorithmiX input for employment contract generation – #AL010FX.

Therefore, pursuant to the above resolution, let it be known that Mr. Vineet Shenoy has been appointed as the CFO of Aretoss Pvt Ltd, effective from 1 June 2025.

______ officer(s) presiding: _______

Officer(s) whose signatures appear on this resolution:

Xavier Bantwal
Dinesh Agarwal
(Digital Signature)
(Digital Signature)

This resolution is the original of a series of resolutions passed


In [8]:
#another test
instr = "Generate a board resolution for the given input."
ui    = "Company Name: Aretoss Pvt Ltd\nDate: 1 June 2025\nPurpose: Appoint CEO\nIndividual Name: Aryan Jadhav"
print(generate_resolution(instr, ui))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 RESOLVED THAT Aryan Jadhav be and is hereby appointed as the Chief Executive Officer of Aretoss Pvt Ltd with immediate effect.

FURTHER RESOLVED THAT Mr. Jadhav's appointment letter be prepared and signed by the Chairperson.

MANY THINGS CONSIDERED: Appointment of CEO

DATED IN INDIA ON THE 1ST DAY OF JUNE, 2025.
__________  (signatures of directors)
